In [1]:
cd("/home/jm2386/Active_Lattice/")
using DrWatson
@quickactivate "Active_Lattice"
include("/home/jm2386/Active_Lattice/src/pm_pdes.jl");
include("/home/jm2386/Active_Lattice/src/pm_sims.jl");
include("/home/jm2386/Active_Lattice/src/pm_plot.jl");
include("/home/jm2386/Active_Lattice/src/Hetrocline.jl");

v3.1


v3.0


v2.1


In [18]:
Params = []
DT, v0, DR, N, Δx, Lx, Ly, δt, δ = (1.0, 7.5, 1.0, 100, 0.05, 20.0, 0.5, 1e-5, 0.1);
# T, save_interval, param_name, pert = (0.1, 0.001, "periodic_solon_test", "lin")
T, save_interval, param_name, pert = (1000.0, 1.0, "periodic_stab_plot", "lin")
Lxs = [Lx]
ϕas = collect(0.35:0.01:0.38)
ϕps = fill(0.3,4)
map(ϕas, ϕps) do ϕa, ϕp
    for Lx in Lxs
        param = new_param(DT, v0, DR, N, Δx, Lx, Ly, ϕa, ϕp, δt, δ; T = T, name = param_name, save_interval = save_interval, save_on = true, pert= pert)
        push!(Params,param)
    end
end
param = Params[4];
map(lin_pert_values, Params)

4-element Vector{Tuple{Float64, ComplexF64, Vector{ComplexF64}}}:
 (0.3141592653589793, -0.010849955436066136 - 0.06798383503896478im, [0.7221429562820239 + 0.0im, 0.6432984709896314 + 0.23497905416914122im, 0.059531888413142844 + 0.07692480925164577im])
 (0.3141592653589793, -0.006829005475576424 + 0.06451710521552835im, [0.714330064609788 + 0.0im, 0.6509962400843904 - 0.23646709846039046im, -0.0575285925335258 + 0.08191597266344722im])
 (0.3141592653589793, -0.0032458722064664515 + 0.06100409556659056im, [0.7065250365469685 + 0.0im, 0.6586885268266452 - 0.23739554248969028im, -0.05543211802054396 + 0.08673196675407957im])
 (0.3141592653589793, -8.210696530211797e-5 + 0.0574795698865091im, [0.6580758122731519 + 0.23486481800684506im, 0.7075207549373004 + 0.0im, -0.08088633397145388 + 0.0681654228568537im])

In [19]:
@unpack Nx = param
using KernelDensitySJ, Peaks

#load saves
ts, f_saves = load_compress_pde(param)
#process
    ft      = permutedims(reshape(reduce(hcat,f_saves),(Nx,3,:)),(3,1,2))
    # ft      = ft[t_start:1:t_end, : ,: ]
    # ts      = ts[t_start:1:end]
    pk      = find_xpeak_ft(ts , ft; time_length = 0.1)
    centre  = Nx ÷ 2 + 1 
    ft  = circshift(ft, (0,-pk + centre, 0));

    t_pde_rgb_image = rho_to_rgb(ft);
    dt, df = t_dff(ts , ft; N=Nx);

    fts = [ft]
    times = [ts]
    images = [t_pde_rgb_image]
    dts = [dt]
    dfs = [df];

full load


saved


In [20]:
New_Params = []
@unpack ϕa, ϕp, Lx = param
DT, v0, DR, N, Δx, Ly, δt, δ = (1.0, 7.5, 1.0, 100, 0.05, 0.5, 1e-5, 0.1);
T, save_interval, param_name, pert = (5000.0, 1.0, "periodic_solon_test", "lin")
Lxs = [2*Lx, 3*Lx, 4*Lx]
for Lx in Lxs
    param = new_param(DT, v0, DR, N, Δx, Lx, Ly, ϕa, ϕp, δt, δ; T = T, name = param_name, save_interval = save_interval, save_on = true, pert= pert)
    push!(New_Params,param)
end
map(lin_pert_values, New_Params)

3-element Vector{Tuple{Float64, ComplexF64, Vector{ComplexF64}}}:
 (0.15707963267948966, 2.752865948312005e-5 - 0.01470071489746831im, [0.6639535875932712 - 0.2332000160113391im, 0.7084520261029836 + 0.0im, 0.041170709553310605 + 0.03441054256750046im])
 (0.10471975511965977, 1.5739074399792414e-5 - 0.006562249093930367im, [0.6650746980977841 - 0.23289229064959183im, 0.7086255659243762 + 0.0im, 0.027541767263097233 + 0.022980106190149913im])
 (0.07853981633974483, 9.521208950659978e-6 + 0.003696955072613197im, [0.6654695953149081 + 0.23278469720986358im, 0.7086863723782016 + 0.0im, -0.020681445489189834 + 0.017245459524967408im])

In [21]:
old_f = f_saves[end]
for (i,param) in enumerate(New_Params)
    f = vcat([old_f for _ in 1:(i+1)]...)
    filename    = pde_save_name(param,0.0)
    data        = Dict("f" => f, "t" => 0.0)
    safesave(filename,data)
end

In [38]:
New_Params = []
DT, v0, DR, N, Δx, Lx, Ly, δt, δ = (1.0, 7.5, 1.0, 100, 0.05, 20.0, 0.5, 1e-5, 0.1);
T, save_interval, param_name, pert = (5000.0, 1.0, "periodic_solon_test", "lin")
Lxs = [2*Lx, 3*Lx, 4*Lx]
ϕas = collect(0.36:0.02:0.38)
ϕps = fill(0.3,3)
map(ϕas, ϕps) do ϕa, ϕp
    for Lx in Lxs
        param = new_param(DT, v0, DR, N, Δx, Lx, Ly, ϕa, ϕp, δt, δ; T = T, name = param_name, save_interval = save_interval, save_on = true, pert= pert)
        push!(New_Params,param)
    end
end

2-element Vector{Nothing}:
 nothing
 nothing

In [39]:
param = New_Params[1]

t, f = load_pde(param,0.0)

(0.0, [0.29385055682518646 0.2985542700969707 0.33077273252132605; 0.29602489222657785 0.299696590810505 0.3280231734767879; … ; 0.28981194883802625 0.2963705431237818 0.3357720965916105; 0.29178209237366576 0.2974460353583561 0.3333521434211934])